In [21]:
from distilabel.llms import OpenAILLM
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadHubDataset, LoadDataFromDicts
from distilabel.steps.tasks import TextGeneration

In [4]:
# !wget https://huggingface.co/datasets/urchade/pile-mistral-v0.1/resolve/main/data.json?download=true -O pile-mistral-v0.1.json

In [24]:
# !pip install openai

  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 9.4 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)


In [6]:
import huggingface_hub
# from google.colab import userdata
huggingface_hub.login(token='<token>') 

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/demiangholipour/.cache/huggingface/token
Login successful


In [13]:
with open('pile-mistral-v0.1.json') as f:
    data = json.load(f)
for x in data:
    x["text"] = " ".join(x["tokenized_text"])
len(data)

19783

In [15]:
data[0]['text']

"Historic shipwreck identified in Lake Michigan MUSKEGON – The bottom of Lake Michigan is literally a graveyard of shipwrecks . Local maritime historians say 1 , 200 of the 2 , 000 sunken vessels in Lake Michigan no longer exist because they hit shore and broke apart . Experts add that about 360 wrecks have been found in the lake ' s deeper water , but there are still many wrecks out there that remain undiscovered . A group of explorers recently found a historic steamship off the coast of Muskegon more than a century after it sank . Members of the Michigan Shipwreck Research Association say this is one of the deepest wrecks ever discovered in Lake Michigan . The story begins around midnight on Feb . 9 , 1899 . Lake Michigan was ice-caked and the 214-foot John V . Moran bucked the ice floes on its run from Milwaukee to Muskegon to deliver a cargo of barreled flour and package goods . The ship was only 11 years old and had an iron-reinforced hull for winter transit , but the conditions o

In [19]:
template = """
Extract all relations between the given entities in the below text, if a relationship exists.
Return your anwer in JSON format with keys `head`, `tail`, and `relation`.\n\nEntities: {ner}\n\nText: {text}
""".strip()

DATA = [
    template.format(**item) for item in data[:10]
]

In [20]:
print(DATA[0])

Extract all relations between the given entities in the below text, if a relationship exists.
Return your anwer in JSON format with keys `head`, `tail`, and `relation`.

Entities: [[139, 142, 'Steamer'], [208, 211, 'Captain'], [126, 130, 'Date'], [151, 151, 'Location'], [6, 6, 'Location'], [90, 90, 'Location'], [153, 153, 'Location'], [4, 5, 'Location'], [11, 12, 'Location'], [35, 36, 'Location'], [117, 118, 'Location'], [132, 133, 'Location'], [138, 138, 'Quantity'], [26, 26, 'Quantity'], [24, 26, 'Quantity'], [29, 31, 'Quantity'], [102, 105, 'Organization']]

Text: Historic shipwreck identified in Lake Michigan MUSKEGON – The bottom of Lake Michigan is literally a graveyard of shipwrecks . Local maritime historians say 1 , 200 of the 2 , 000 sunken vessels in Lake Michigan no longer exist because they hit shore and broke apart . Experts add that about 360 wrecks have been found in the lake ' s deeper water , but there are still many wrecks out there that remain undiscovered . A group

In [48]:
llm = OpenAILLM(model="gpt-3.5-turbo")

In [51]:
dir(llm)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_validator__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__'

In [22]:
with Pipeline(
    name="simple-text-generation-pipeline",
    description="A simple text generation pipeline",
) as pipeline:
    # load_dataset = LoadHubDataset(
    #     name="load_dataset",
    #     output_mappings={"prompt": "instruction"},
    # )
    load_dataset = LoadDataFromDicts(
        name="load_data",
        data=[
            {
                "instruction": sample,
            } for sample in DATA
        ],
        batch_size=1,
    )
    generate_with_openai = TextGeneration(
        name="generate_with_gpt35", llm=OpenAILLM(model="gpt-3.5-turbo")
    )
    load_dataset.connect(generate_with_openai)


In [25]:
distiset = pipeline.run(
    parameters={
        # "load_dataset": {
        #     "repo_id": "distilabel-internal-testing/instruction-dataset-mini",
        #     "split": "test",
        # },
        "generate_with_gpt35": {
            "llm": {
                "generation_kwargs": {
                    "temperature": 0.7,
                    "max_new_tokens": 512,
                }
            }
        },
    },
)

[04/23/24 14:59:36] INFO     ['distilabel.pipeline.local'] 📝 Pipeline data will be written to         ]8;id=53167;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=387488;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py#113\113]8;;\
                             '/home/demiangholipour/.cache/distilabel/pipelines/b19c2d04bd92b8faf8e30d             
                             272f875f9c2fc27e21/data'                                                              

                    INFO     ['distilabel.pipeline.local'] ⏳ Waiting for all the steps to load...     ]8;id=593054;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=873749;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py#366\366]8;;\

[04/23/24 14:59:39] INFO     ['distilabel.pipeline.local'] ⏳ Steps loaded: 2/2                        ]8;id=25789;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=193525;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py#380\380]8;;\

                    INFO     ['distilabel.pipeline.local'] ✅ All the steps have been loaded!          ]8;id=244002;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=582872;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py#384\384]8;;\

[04/23/24 14:59:41] INFO     ['distilabel.step.load_data'] 🧬 Starting yielding batches from generator ]8;id=374984;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=56581;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py#747\747]8;;\
                             step 'load_data'. Offset: 0                                                           

...

                    INFO     ['distilabel.step.load_data'] 🏁 Finished running step 'load_data'        ]8;id=88543;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=923848;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py#715\715]8;;\

                    INFO     ['distilabel.step.generate_with_gpt35'] 📦 Processing batch 0 in          ]8;id=387509;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=944762;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py#792\792]8;;\
                             'generate_with_gpt35'                                                                 

[04/23/24 14:59:56] INFO     ['distilabel.step.generate_with_gpt35'] 📨 Step 'generate_with_gpt35'     ]8;id=664315;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=659543;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py#825\825]8;;\
                             sending batch 0 to output queue                                                       

                    INFO     ['distilabel.step.generate_with_gpt35'] 🏁 Finished running step          ]8;id=192319;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py\local.py]8;;\:]8;id=577926;file:///home/demiangholipour/miniconda3/envs/glirel/lib/python3.10/site-packages/distilabel/pipeline/local.py#715\715]8;;\
                             'generate_with_gpt35'                                                                 

Generating train split: 10 examples [00:00, 3070.05 examples/s]


In [36]:
distiset

Distiset({
    default: DatasetDict({
        train: Dataset({
            features: ['instruction', 'model_name', 'generation'],
            num_rows: 10
        })
    })
})

In [47]:
for x in distiset['default']['train']:
    print(x['generation'])

```json
[
    {
        "head": "Steamer",
        "tail": "Lake Michigan",
        "relation": "located_in"
    },
    {
        "head": "Captain",
        "tail": "John McLeod",
        "relation": "has_captain"
    },
    {
        "head": "Date",
        "tail": "Feb. 9, 1899",
        "relation": "sunk_on"
    },
    {
        "head": "Quantity",
        "tail": "214-foot",
        "relation": "has_length"
    },
    {
        "head": "Quantity",
        "tail": "11 years old",
        "relation": "has_age"
    },
    {
        "head": "Organization",
        "tail": "Michigan Shipwreck Research Association",
        "relation": "member_of"
    }
]
```
```json
[
    {
        "head": "Organization",
        "tail": "Organization",
        "relation": "co-occurs"
    },
    {
        "head": "Organization",
        "tail": "Organization",
        "relation": "co-occurs"
    },
    {
        "head": "Classification",
        "tail": "Classification",
        "relation": "related"
  